In [12]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./fastapi.txt")
text_document =loader.load()
text_document

[Document(page_content='FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.6+ based on standard Python type hints.\nFeatures:\nFast: FastAPI is very fast, on par with Node.js and Go (thanks to Starlette and Pydantic).\nFastAPI uses ASGI for high performance.\nFast to code: with automatic type checking and IDE support, FastAPI is much faster to code than other Python frameworks.\nEasy to learn: FastAPI is easy to learn, even for beginners.\nEasy to use: FastAPI is very easy to use, with a small API surface and many features.\nInstallation:\n\nTo install FastAPI, you can use pip:\n\npip install fastapi\n\n\nUsage:\nTo create a FastAPI app, you can use the following code:\n\n\nfrom fastapi import FastAPI\n\napp = FastAPI()\nThen, you can define your endpoints:\nPython\n\n\n\n@app.get("/")\nasync def root():\n    return {"message": "Hello World"}\n\nFinally, you can run your app:\n\nuvicorn main:app --reload\n\n\nExample:\nHere is an example of a Fa

In [13]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader = WebBaseLoader("https://fastapi.tiangolo.com/",bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("md-content"))),
                       )
text_document=loader.load()
text_document

[Document(page_content='\n\nFastAPI\n\n\n\n\n\nFastAPI framework, high performance, easy to learn, fast to code, ready for production\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDocumentation: https://fastapi.tiangolo.com\nSource Code: https://github.com/tiangolo/fastapi\n\nFastAPI is a modern, fast (high-performance), web framework for building APIs with Python based on standard Python type hints.\nThe key features are:\n\nFast: Very high performance, on par with NodeJS and Go (thanks to Starlette and Pydantic). One of the fastest Python frameworks available.\nFast to code: Increase the speed to develop features by about 200% to 300%. *\nFewer bugs: Reduce about 40% of human (developer) induced errors. *\nIntuitive: Great editor support. Completion everywhere. Less time debugging.\nEasy: Designed to be easy to use and learn. Less time reading docs.\nShort: Minimize code duplication. Multiple features from each parameter declaration. Fewer bugs.\nRobust: Get production-ready code. With automatic 

In [14]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./fastapi_tutorial.pdf")
docs = loader.load()
docs

[Document(page_content='  \n  ', metadata={'source': './fastapi_tutorial.pdf', 'page': 0}),
 Document(page_content='FastAPI – Python Web Framework  \n \ni \n \n \nAbout the T utorial  \nFastAPI is a modern Python web framework, very efficient in building APIs. \nFastAPI has been developed by Sebastian Ramirez in Dec. 2018. FastAPI \n0.68.0 is the currently available version. The latest version requires Python \n3.6 or above.  It is one of the fastest web frameworks of Python.  \nAudience  \nThis tutorial is designed for developers who want to learn how to build REST \nAPIs using Python.  \nPrerequisites  \nBefore you proceed, make sure that you understand the basics of procedural \nand object -oriented programming in Python. Knowledge of REST \narchitecture is an added advantage.  \nDisclaimer  & Copyright  \n\uf0e3 Copyright 2022  by Tutorials Point (I) Pvt. Ltd.  \nAll the conte nt and graphics published in this e -book are the property of \nTutorials Point (I) Pvt. Ltd. The user of 

In [15]:
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load example document


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=20,
    
)
documents=text_splitter.split_documents(docs)


In [17]:
documents

[Document(page_content='FastAPI – Python Web Framework  \n \ni \n \n \nAbout the T utorial  \nFastAPI is a modern Python web framework, very efficient in building APIs. \nFastAPI has been developed by Sebastian Ramirez in Dec. 2018. FastAPI \n0.68.0 is the currently available version. The latest version requires Python \n3.6 or above.  It is one of the fastest web frameworks of Python.  \nAudience  \nThis tutorial is designed for developers who want to learn how to build REST \nAPIs using Python.  \nPrerequisites  \nBefore you proceed, make sure that you understand the basics of procedural \nand object -oriented programming in Python. Knowledge of REST \narchitecture is an added advantage.  \nDisclaimer  & Copyright  \n\uf0e3 Copyright 2022  by Tutorials Point (I) Pvt. Ltd.  \nAll the conte nt and graphics published in this e -book are the property of \nTutorials Point (I) Pvt. Ltd. The user of this e -book is prohibited to reuse, \nretain, copy, distribute or republish any contents or

In [18]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, OllamaEmbeddings())

In [19]:
db

In [20]:
from langchain_community.llms import Ollama
llm=Ollama(model="mistral")
llm

Ollama(model='mistral')

In [23]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt =("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful.
{context}
""")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm, prompt)

In [29]:
#retriever
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002462A77B140>)

In [31]:
from langchain.chains import create_retrieval_chain

retriever_chain = create_retrieval_chain(retriever, document_chain)

In [34]:
response=retriever_chain.invoke({"input":"why is fastapi faster"})

In [36]:
response['answer']

' FastAPI is known to be faster than other Python web frameworks for several reasons:\n\n1. Asynchronous Processing: FastAPI uses an asynchronous architecture which allows multiple requests to be handled concurrently, improving the overall performance of the application.\n2. Built-in support for modern standards: FastAPI has built-in support for modern standards such as ASGI and OpenAPI, which help in faster development and deployment.\n3. Pydantic models: FastAPI uses Pydantic models for input validation, data parsing, and response serialization. This results in fewer lines of code, less boilerplate, and faster processing times.\n4. Type hinting: FastAPI supports type hinting which allows the framework to understand the data types being used and improve its performance accordingly.\n5. Streamlined routing: FastAPI has a streamlined routing system that makes it easier and faster to define routes and endpoints for your application.\n6. Middleware: FastAPI provides a robust middleware sy